# Assigment - Weakly interacting and confined bosons at low density

### _Authors:_ Clàudia Garcia and Adrià Rojo

L'equació de Gross-Pitaevskii (GP) ens permet descriure els experiments de condensats de Bose-Einstein dins del marc tèoric. 

En aquest exemple, considerem N àtoms de $^{87}$ Rb, on tots els àtoms estàn confinats per un camp magnètic, i tots els efectes d'aquest es descriuen bé per un potencial harmònic. També considerem que $\Psi(\vec{r})$ està normalitzat a la unitat. L'equació GP és la següent:

$$
\left[-\frac{1}{2} \nabla^2+\frac{1}{2} r_1^2+4 \pi \bar{a}_s N\left|\bar{\Psi}\left(\vec{r}_1\right)\right|^2\right] \bar{\Psi}\left(\vec{r}_1\right)=\bar{\mu} \bar{\Psi}\left(\vec{r}_1\right),
$$

on $\mu$ és el potencial químic, $N$ el nombre de partícules i $\bar{a}_s$ és la longitud de $s$-wave scattering. L'equació està escrita en unitats de oscil·lador harmònic.

Començarem per precomputar valors de distància i variables de la simulació i la funció d'ona inicial i fer tots els `imports`. Per la segona derivada utilitzarem la funció `gradient` de `numpy`. Utilitzarem `tqdm` per iterar el bucle i tenir una estimació del temps d'execució. El codi utilitza de forma extensiva generadors, per evitar l'anidament en un segon bucle `for`. 

In [16]:
import numpy as np
import numpy.typing as npt
from tqdm import tqdm
from IPython.display import display, Math, Markdown

def second_derivative(fun: npt.ArrayLike, h: float) -> npt.ArrayLike:
    first = np.gradient(fun, h)
    return np.gradient(first, h)

def final_properties(psi, local_mu, step, rs, r2s, interaction):
    psi_dx2 = second_derivative(psi, step)

    local_values = [
        (r2*final*final, #acc radius
            final*dx2, # kin energy
            r2*final*final, # armonic
            r2*(final/r)**4 if r != 0 else 0, # interaction energy
            mu*final*final, # average chem potential
            r2/2 + interaction * (final/r)**2 if r != 0 else 0, # particle potential
            (final/r)**2 if r != 0 else 0 # density
        )
        for r, r2, final, dx2, mu in zip(rs, r2s, psi, psi_dx2, local_mu)
    ]

    local_accumulted_radius, local_kinetic_energy, local_armonic_potential, local_interaction_energy, local_average_chem_potential, particle_potential, density =\
        zip(*local_values)

    squared_radius = sum(local_accumulted_radius)*step
    mean_squared_radius = np.sqrt(squared_radius)
    average_chem_potential = sum(local_average_chem_potential)*step
    kinetic_energy = -sum(local_kinetic_energy)*step/2
    armonic_potential = sum(local_armonic_potential)*step/2
    interaction_energy = sum(local_interaction_energy)*step*interaction/2

    return {
        "r^2": squared_radius,
        "<r^2>": mean_squared_radius,
        "avg mu": average_chem_potential,
        "kin energy": kinetic_energy,
        "armonic pot" : armonic_potential,
        "interac energy": interaction_energy,
        "total energy": kinetic_energy+armonic_potential+interaction_energy
    }

def initial_values(scattering_length, width, step, atom_numbers, alpha):
    alpha2 = alpha*alpha
    cvar = 2 * np.power(alpha, 3/2)/ np.power(np.pi, 1/4)
    rs = np.array([i * step for i in range(width)])
    r2s = rs**2
    psi = np.array([cvar*r*np.exp(-0.5*alpha2*r2) for (r, r2) in zip(rs, r2s)])
    interaction = scattering_length*atom_numbers
    density_param = atom_numbers*scattering_length*scattering_length*scattering_length
    return alpha2, cvar, interaction, density_param,rs, r2s, psi
    
def compute_wavelength(rs, r2s, psi, iterations, step, interaction, time_step, label=None):
    psi_curr = psi
    for i in tqdm(range(iterations), desc=label):
        psi_dx2 = second_derivative(psi_curr, step)
        # calculo de energia y mu local
        local_energy_mu = [(-(curr*dx2+r2*curr**2+interaction*r2*(curr/r)**4)/2 if r != 0 else 0, 
                            ((-dx2/curr) + r2 + interaction*(curr/r)**2)/2 if r != 0 else 0)
                        for r, r2, curr, dx2 in zip(rs, r2s, psi_curr, psi_dx2)]
        # deshacer la lista de tuplas (energía y mu de cada distancia) 
        # en una tupla de listas (lista de todas las energías y lista de todas las mus)
        local_energy, local_mu = zip(*local_energy_mu)
        energy = sum(local_energy)*step
        psi_next = [curr*(1 - time_step*mu) for curr, mu in zip(psi_curr, local_mu)]
        normalization_const = np.sqrt(sum(next*next for next in psi_next)*step)
        psi_curr = np.array(psi_next)/normalization_const
    
    return (psi_curr, local_mu)

### a0) Terme d'interacció nul.

Si considerem el terme d'interacció nul (en el nostre programa la variable `interaction`), l'equació GP queda com: 

$$
\left[-\frac{1}{2} \nabla^2+\frac{1}{2} r_1^2\right] \bar{\Psi}\left(\vec{r}_1\right)=\bar{\mu} \bar{\Psi}\left(\vec{r}_1\right),
$$

Que com veiem
$
H_{o.h} = -\frac{1}{2} \nabla^2+\frac{1}{2} r_1^2,
$
 coincideix amb l'expressió de l'hamiltonià de l'oscil·lador harmònic en tres dimensions, per tant, tenim que: 
$H_{o.h} \bar{\Psi}\left(\vec{r}_1\right) = \varepsilon \bar{\Psi}\left(\vec{r}_1\right)$ i comparant ambdós equacions obtenim que $\bar{\mu} = \varepsilon$, és a dir, el potencial químic coincideix amb l'energia per partícula ($ \varepsilon = E/N$).

In [17]:
scattering_length, width,  step, atom_numbers, time_step, alpha, iterations = \
          0.00433,  400,  0.015,        10000,    0.0001,   0.8,      70000

alpha2, cvar, interaction, density_param, rs, r2s, psi = \
    initial_values(scattering_length, width, step, atom_numbers, alpha)
interaction = 0

final, local_mu = compute_wavelength(rs, r2s, psi, iterations,step, interaction,time_step)

properties = final_properties(final, local_mu, step, rs, r2s, interaction)

display(Math("E = {total energy:.5f}, \\mu = {avg mu:.5f}".format(**properties)))
display(Math("\\varepsilon_{{kin}} = {kin energy:.5f}, \\varepsilon_{{o.h}} = {armonic pot:.5f}, \\varepsilon_{{int}} = {interac energy:.5f}".format(**properties)))

100%|██████████| 70000/70000 [00:33<00:00, 2085.38it/s]


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Podem observar que tant l'energia cinética com la de l'oscil·lador harmònic es reparteixen de forma igual l'energia, complint d'aquesta forma el teorema del virial per aquests casos. Evidentment, el terme d'energia d'interacció és igual a zero.

### a) Resolució de l'equació GP per a diferents $N$

Utilizant $\bar{a}_s = 0.00433$ resolem l'equació GP per diferents nombres de partícules ($N$) i obtenim els següents resultats per $\mu$ i les següents energies per partícules: total, cinètica, de l'oscil·lador harmònic i d'interacció.



In [ ]:
markdown_table = "| $N$   | $\\bar{\\mu}$ | $\\varepsilon$ | $\\varepsilon_{kin}$ | $\\varepsilon_{o.h}$ | $\\varepsilon_{int}$ |\n" +\
                 "|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|\n"
for particles in [100, 1000, 10000, 100000, 1000000]:
    scattering_length, width,   step, atom_numbers, time_step, alpha, iterations = \
              0.00433,   400,  0.015,    particles,    0.0001,   0.8,      40000

    alpha2, cvar, interaction, density_param, rs, r2s, psi = \
        initial_values(scattering_length, width, step, atom_numbers, alpha)
    final, local_mu = compute_wavelength(rs, r2s, psi, iterations,step, interaction,time_step, label="N = {0:7d}".format(particles))

    properties = final_properties(final, local_mu, step, rs, r2s, interaction)
    markdown_table += "|{particles}|{avg mu:.5f}|{total energy:.5f}|{kin energy:.5f}|{armonic pot:.5f}|{interac energy:.5f}|\n".format(particles=particles, **properties)

display(Markdown(markdown_table))


Particles 1000000: 100%|██████████| 40000/40000 [00:18<00:00, 2132.39it/s]


| $N$   | $\bar{\mu}$ | $\varepsilon$ | $\varepsilon_{kin}$ | $\varepsilon_{o.h}$ | $\varepsilon_{int}$ |
|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|
|100|1.65589|1.65589|0.69589|0.80908|0.15092|
|1000|2.48689|2.48689|0.50978|1.13860|0.83851|
|10000|5.29692|5.29692|0.29058|2.31160|2.69474|
|100000|12.81550|12.81550|0.15246|5.51201|7.15103|
|1000000|40.98343|40.98343|0.02722|10.03987|30.91634|


Observem que l'energia d'interacció augmenta amb la quantitat de partícules i en canvi l'energía cinética disminueix.

### b) Aproximació de Thomas-Fermi.
 En el marc d'aquesta aproximació el terme cinètic es pot considerar nul i l'equació GP queda com: 

$$
\left[\frac{1}{2} r_1^2+4 \pi \bar{a}_s N\left|\bar{\Psi}\left(\vec{r}_1\right)\right|^2\right] \bar{\Psi}\left(\vec{r}_1\right)=\bar{\mu} \bar{\Psi}\left(\vec{r}_1\right)
$$


In [4]:
def initial_values_TF(scattering_length, width, step, atom_numbers):
    rs = np.array([i * step for i in range(width)])
    r2s = rs**2
    g_1D = scattering_length * atom_numbers  # Interacción efectiva en 1D
    mu_TF = (15 * g_1D / (8 * np.sqrt(2 * np.pi)))**(2/5)  # Potencial químico TF (ajustar según sistema)
    psi_TF = np.array([np.sqrt(max((mu_TF - 0.5 * r2) / g_1D, 0)) for r2 in r2s])
    return rs, r2s, psi_TF

def compute_wavelength_TF(rs, r2s, psi, iterations, step, interaction, time_step, label=None):
    psi_curr = psi
    for i in tqdm(range(iterations), desc=label):
        # Eliminar cálculo de derivadas (no hay energía cinética)
        local_mu = [(r2 + interaction * (curr / r)**2) / 2 if r != 0 else 0 
                    for r, r2, curr in zip(rs, r2s, psi_curr)]
        psi_next = [curr * (1 - time_step * mu) for curr, mu in zip(psi_curr, local_mu)]
        normalization_const = np.sqrt(sum(next**2 for next in psi_next) * step)
        psi_curr = np.array(psi_next) / normalization_const
    return psi_curr, local_mu

def final_properties_TF(psi, rs, r2s, interaction):
    density = np.array([(curr / r)**2 if r != 0 else 0 for r, curr in zip(rs, psi)])
    armonic_potential = sum(0.5 * r2 * curr**2 for r2, curr in zip(r2s, psi)) * step
    interaction_energy = 0.5 * interaction * sum(r2 * (curr / r)**4 if r != 0 else 0 
                        for r, r2, curr in zip(rs, r2s, psi)) * step
    return {
        "armonic pot": armonic_potential,
        "interac energy": interaction_energy,
        "total energy": armonic_potential + interaction_energy,
        "density": density
    }

In [9]:
markdown_table = "| $N$   | $\\bar{\\mu}$ | $\\varepsilon$ | $\\varepsilon_{kin}$ | $\\varepsilon_{o.h}$ | $\\varepsilon_{int}$ |\n" +\
                 "|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|\n"

for particles in [100, 1000, 10000, 100000, 1000000]:
    scattering_length, width,  step, atom_numbers, time_step, alpha, iterations = \
          0.00433,  400,  0.015,        10000,    0.0001,   0.8,      40000
    interaction = scattering_length*atom_numbers

    rs, r2s, psi = initial_values_TF(scattering_length, width, step, particles)
    psi, local_mu = compute_wavelength_TF(rs, r2s, psi, iterations, step, interaction, time_step, label="N = {0:7d}".format(particles))
    properties = final_properties_TF(psi, rs, r2s, interaction)
    markdown_table += "|{particles}|{total energy:.5f}|{armonic pot:.5f}|{interac energy:.5f}|\n".format(particles=particles, **properties)

display(Markdown(markdown_table))


N = 1000000: 100%|██████████| 40000/40000 [00:08<00:00, 4542.20it/s]


| $N$   | $\bar{\mu}$ | $\varepsilon$ | $\varepsilon_{kin}$ | $\varepsilon_{o.h}$ | $\varepsilon_{int}$ |
|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|
|100|6414814.81493|0.00011|6414814.81481|
|1000|6414814.81493|0.00011|6414814.81481|
|10000|0.00030|0.00025|0.00005|
|100000|0.00034|0.00028|0.00006|
|1000000|0.00035|0.00029|0.00006|


### c) Plots de la de densitat $\rho(r_1)$ per $N = 1000$ i $N = 100000$. 

Compararem els resultats per l'equació GP i per aquesta en l'aproximació de TF. 

La densitat $\rho(r_1)$ està normalitzada com: $\int dr_1 r_1^2 \rho(r_1)$

En el cas de l'aproximació TF tenim la següent expressió per la densitat: 
$$
\rho(r_1) = \frac{1}{4 \pi N \bar{a}_s} \cdot \left( \bar{\mu} - \frac{1}{2}r_1^2 \right)
$$

### d) Teorema del Virial per diferents valors de $N$

Comprovarem que les solucions obtingudes de l'equació GP compleix el Teorema del Virial. 

El teorema del Virial en el marc de l'equació GP és de la forma: 
$$
2\varepsilon_{kin} - 2\varepsilon_{o.h} + 3\varepsilon_{int} = 0
$$

| $N$   | $2\varepsilon_{kin} - 2\varepsilon_{o.h} + 3\varepsilon_{int}$ | $- 2\varepsilon_{o.h}^{TF} + 3\varepsilon_{int}^{TF}$ | 
|:-----:|:-----------:|:-------------:|
|100    |             |     |
|1000   |
|10000  |
|100000 |
|1000000|

comment behaviour...